<a href="https://colab.research.google.com/github/ReisK1ng/Projeto404/blob/main/ProjetoBanco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aluno: Rafael Leonardo dos Reis Freitas

RGM: 37743449

Curso: Ciência da Computação

Turma: D1

Link do repositório Git


In [ ]:
import sqlite3
import os
import re
from abc import ABC, abstractmethod
from collections.abc import Iterable, Sized

# === Exceções Personalizadas ===
class ErroDeConta(Exception):
    # Exceção base para o sistema bancário
    pass

class SaldoInsuficienteError(ErroDeConta):
    # Exceção para saldo insuficiente
    pass

class OperacaoInvalidaError(ErroDeConta):
    # Exceção para operação inválida
    pass

class ContaNaoEncontradaError(ErroDeConta):
    # Exceção quando a conta não é encontrada
    pass

# === Classe Cliente ===
class Cliente:
    def __init__(self, nome, cpf):
        # Inicializa cliente com nome e CPF validado
        if not re.fullmatch(r"\d{11}", cpf):
            raise OperacaoInvalidaError("CPF inválido: deve conter 11 dígitos")
        self.__nome = nome
        self.__cpf = cpf

    def __str__(self):
        return f'{self.__nome} ({self.__cpf})'

    @property
    def nome(self): return self.__nome

    @property
    def cpf(self): return self.__cpf

# === Classe Abstrata Conta ===
class Conta(ABC):
    _total_contas = 0  # Contador de contas

    def __init__(self, numero, cliente, saldo=0.0):
        if not isinstance(cliente, Cliente):
            raise OperacaoInvalidaError("Cliente inválido")
        self._numero = numero
        self._cliente = cliente
        self.__saldo = max(saldo, 0.0)
        Conta._total_contas += 1

    @property
    def numero(self): return self._numero

    @property
    def cliente(self): return self._cliente

    @property
    def saldo(self): return self.__saldo

    @saldo.setter
    def saldo(self, valor):
        if valor < 0:
            raise OperacaoInvalidaError("Saldo não pode ser negativo")
        self.__saldo = valor

    @classmethod
    def total_contas(cls):
        return cls._total_contas

    def depositar(self, valor):
        if valor <= 0:
            raise OperacaoInvalidaError("Valor de depósito deve ser positivo")
        self.__saldo += valor

    def sacar(self, valor):
        self._validar_saque(valor)
        self.saldo -= valor

    def _validar_saque(self, valor):
        if valor <= 0:
            raise OperacaoInvalidaError("Valor de saque deve ser positivo")
        if valor > self.saldo:
            raise SaldoInsuficienteError("Saldo insuficiente")

    def transferir(self, destino, valor):
        if not isinstance(destino, Conta):
            raise OperacaoInvalidaError("Conta de destino inválida")
        self.sacar(valor)
        destino.depositar(valor)

    def extrato(self):
        return f'Conta {self._numero} - {self._cliente.nome} - Saldo: R${self.__saldo:.2f}'

    @abstractmethod
    def atualiza(self, taxa):
        pass

    def atualiza(self, taxa):
        if taxa < 0:
            raise OperacaoInvalidaError("Taxa não pode ser negativa")
        self.__saldo += self.__saldo * taxa

# === Conta Corrente ===
class ContaCorrente(Conta):
    def sacar(self, valor):
        self._validar_saque(valor)
        self.saldo -= valor

    def depositar(self, valor):
        super().depositar(valor - 0.10)

    def atualiza(self, taxa):
        super().atualiza(taxa * 2)

# === Conta Poupança ===
class ContaPoupanca(Conta):
    def sacar(self, valor):
        self._validar_saque(valor)
        self.saldo -= valor

    def atualiza(self, taxa):
        super().atualiza(taxa * 3)

# === Banco ===
class Banco(Iterable, Sized):
    def __init__(self, db_name="banco.db"):
        self.__db_name = db_name
        self.__setup_database()

    def __setup_database(self):
        with sqlite3.connect(self.__db_name) as conn:
            cursor = conn.cursor()

            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='contas'")
            tabela_antiga = cursor.fetchone()

            if tabela_antiga:
                cursor.execute("PRAGMA table_info(contas)")
                colunas = [info[1] for info in cursor.fetchall()]
                if 'cpf_cliente' not in colunas:
                    cursor.execute("DROP TABLE IF EXISTS clientes")
                    cursor.execute("DROP TABLE IF EXISTS contas")
                    conn.commit()

            cursor.execute("""
                CREATE TABLE IF NOT EXISTS clientes (
                    cpf TEXT PRIMARY KEY,
                    nome TEXT NOT NULL
                )
            """)
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS contas (
                    numero INTEGER PRIMARY KEY,
                    cpf_cliente TEXT NOT NULL,
                    tipo TEXT NOT NULL,
                    saldo REAL NOT NULL,
                    FOREIGN KEY (cpf_cliente) REFERENCES clientes(cpf)
                )
            """)
            conn.commit()

    def __conectar(self):
        return sqlite3.connect(self.__db_name)

    def adicionar_conta(self, conta):
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT 1 FROM clientes WHERE cpf = ?", (conta.cliente.cpf,))
            if not cursor.fetchone():
                cursor.execute(
                    "INSERT INTO clientes (cpf, nome) VALUES (?, ?)",
                    (conta.cliente.cpf, conta.cliente.nome)
                )

            cursor.execute(
                "INSERT INTO contas (numero, cpf_cliente, tipo, saldo) VALUES (?, ?, ?, ?)",
                (conta.numero, conta.cliente.cpf, conta.__class__.__name__, conta.saldo)
            )
            conn.commit()

    def buscar_conta(self, numero):
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT c.numero, cl.nome, cl.cpf, c.tipo, c.saldo
                FROM contas c
                JOIN clientes cl ON c.cpf_cliente = cl.cpf
                WHERE c.numero = ?
            """, (numero,))
            dados = cursor.fetchone()

            if not dados:
                raise ContaNaoEncontradaError(f"Conta {numero} não encontrada")

            cliente = Cliente(dados[1], dados[2])
            if dados[3] == "ContaCorrente":
                return ContaCorrente(dados[0], cliente, dados[4])
            elif dados[3] == "ContaPoupanca":
                return ContaPoupanca(dados[0], cliente, dados[4])
            else:
                raise OperacaoInvalidaError(f"Tipo de conta inválido: {dados[3]}")

    def listar_contas(self):
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT c.numero, cl.nome, cl.cpf, c.tipo, c.saldo
                FROM contas c
                JOIN clientes cl ON c.cpf_cliente = cl.cpf
                ORDER BY c.numero
            """)
            return cursor.fetchall()

    def atualizar_saldo(self, numero, saldo):
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute(
                "UPDATE contas SET saldo = ? WHERE numero = ?",
                (saldo, numero)
            )
            conn.commit()

    def remover_conta(self, numero):
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("DELETE FROM contas WHERE numero = ?", (numero,))
            conn.commit()

    def __iter__(self):
        contas = self.listar_contas()
        for conta_data in contas:
            yield self.buscar_conta(conta_data[0])

    def __len__(self):
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM contas")
            return cursor.fetchone()[0]

# === Menu CLI ===
def menu():
    banco = Banco()

    while True:
        print("\n==== MENU BANCÁRIO ====")
        print("1. Criar conta")
        print("2. Depositar")
        print("3. Sacar")
        print("4. Transferir")
        print("5. Listar contas")
        print("6. Remover conta")
        print("7. Sair")

        try:
            opcao = input("Escolha uma opção: ").strip()

            match opcao:
                case "1":
                    nome = input("Nome do cliente: ").strip()
                    cpf = input("CPF (11 dígitos): ").strip()
                    tipo = input("Tipo (corrente/poupança): ").strip().lower()
                    numero = int(input("Número da conta: "))
                    saldo_inicial = float(input("Saldo inicial: R$ "))

                    cliente = Cliente(nome, cpf)
                    if tipo == "corrente":
                        conta = ContaCorrente(numero, cliente, saldo_inicial)
                    elif tipo == "poupança":
                        conta = ContaPoupanca(numero, cliente, saldo_inicial)
                    else:
                        print("Tipo de conta inválido!")
                        continue

                    banco.adicionar_conta(conta)
                    print(f"Conta {numero} criada com sucesso!")

                case "2":
                    numero = int(input("Número da conta: "))
                    valor = float(input("Valor a depositar: R$ "))
                    conta = banco.buscar_conta(numero)
                    conta.depositar(valor)
                    banco.atualizar_saldo(numero, conta.saldo)
                    print(f"Depósito de R$ {valor:.2f} realizado com sucesso!")

                case "3":
                    numero = int(input("Número da conta: "))
                    valor = float(input("Valor a sacar: R$ "))
                    conta = banco.buscar_conta(numero)
                    conta.sacar(valor)
                    banco.atualizar_saldo(numero, conta.saldo)
                    print(f"Saque de R$ {valor:.2f} realizado com sucesso!")

                case "4":
                    origem = int(input("Conta de origem: "))
                    destino = int(input("Conta de destino: "))
                    valor = float(input("Valor a transferir: R$ "))
                    conta_origem = banco.buscar_conta(origem)
                    conta_destino = banco.buscar_conta(destino)
                    conta_origem.transferir(conta_destino, valor)
                    banco.atualizar_saldo(origem, conta_origem.saldo)
                    banco.atualizar_saldo(destino, conta_destino.saldo)
                    print(f"Transferência de R$ {valor:.2f} realizada com sucesso!")

                case "5":
                    print("\n=== LISTA DE CONTAS ===")
                    for conta_data in banco.listar_contas():
                        print(f"Conta {conta_data[0]} - {conta_data[1]} ({conta_data[2]})")
                        print(f"  Tipo: {conta_data[3]} - Saldo: R$ {conta_data[4]:.2f}")
                    print("======================")

                case "6":
                    numero = int(input("Número da conta a remover: "))
                    banco.remover_conta(numero)
                    print(f"Conta {numero} removida com sucesso!")

                case "7":
                    print("Encerrando sistema bancário...")
                    break

                case _:
                    print("Opção inválida! Tente novamente.")

        except ValueError:
            print("Erro: Valor inválido fornecido!")
        except SaldoInsuficienteError as e:
            print(f"Erro: {e}")
        except OperacaoInvalidaError as e:
            print(f"Erro: {e}")
        except ContaNaoEncontradaError as e:
            print(f"Erro: {e}")
        except Exception as e:
            print(f"Erro inesperado: {e}")

if __name__ == "__main__":
    menu()